<a href="https://colab.research.google.com/github/omkar1kadam/Fake_news_detection/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow pandas numpy scikit-learn matplotlib


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [3]:
true_news = pd.read_csv("True.csv")
fake_news = pd.read_csv("Fake.csv")
print(true_news.head())
print("thenn")
print(fake_news.head())

FileNotFoundError: [Errno 2] No such file or directory: 'True.csv'

In [ ]:
true_news['label'] = 1
fake_news['label'] = 0

In [ ]:
data = pd.concat([true_news, fake_news])

data = data.sample(frac=1).reset_index(drop=True)

print(data.head())

In [ ]:
X = data['text']
y = data['label']

In [ ]:
vocab_size = 10000
oov_token = "<OOV>"

Tokenizer = Tokenizer(num_words=vocab_size , oov_token = oov_token)
Tokenizer.fit_on_texts(X)

Sequences = Tokenizer.texts_to_sequences(X)

print(Sequences[0])

In [ ]:
max_length = 300
padded_sequences = pad_sequences(Sequences,maxlen = max_length,padding="post" , truncating= "post")

print(padded_sequences[0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y , test_size= 0.2, random_state=42)

In [ ]:
model = Sequential([
    Embedding(vocab_size, 64, input_length=max_length),
    Bidirectional(LSTM(64)),  # This line needs the comma at the end
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
def predict_news(news_text):
    sequence = tokenizer.texts_to_sequences([news_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded)[0][0]
    return "Real" if prediction > 0.5 else "Fake"

print(predict_news("The government announced a new plan to help small businesses today."))
